<a href="https://colab.research.google.com/github/scarlettyu2023/AI_agent_workshop/blob/main/Topic_3_Agent_Tool_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Goals
Know how and why to run an Ollama LLM server

Know how to run large commerical models in LangGraph

Understand how LLMs call tools using the OpenAI format

Understand how to most efficiently define and use LangGraph tools

#1. Set up an Ollama server running Llama 3.2-1B  on either your laptop (if it has a GPU) or on CoLab (if not).  

Create two copies of the  llama_mmlu_eval.py code from Topic 1 and modify them so that each runs on a single different topic.  Use the command line time shell function  to measure how long each takes to run in real clock time.  Then modify the programs so they use Ollama.  Start the Ollama server running and then time both sequential and parallel execution of the two programs.  For sequential execution, run

time { python program1.py ; python program2.py }
Next, time them running in parallel:

time { python program1.py & python program2.py & wait; }
Write in the README.md file in your portfolio what you observed.



In [8]:
!sudo apt-get update -qq
!sudo apt-get install -y zstd

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 2s (341 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf:

In [11]:
%%bash
set -e
sudo pkill -f "ollama" || true
sudo fuser -k 11434/tcp || true
sleep 1
ps aux | grep -i ollama | head -n 20


root        6420  0.0  0.0   6480  2516 ?        S    19:46   0:00 grep -i ollama


In [12]:
%%bash
set -e
nohup ollama serve > ollama.log 2>&1 &
sleep 2

# Check it is running
ps aux | grep -i "ollama serve" | grep -v grep || true

# Check port is open
curl -s http://127.0.0.1:11434/api/tags | head -n 5 || true

echo "---- last 30 lines of ollama.log ----"
tail -n 30 ollama.log


root        6507  1.0  0.2 1784652 32292 ?       Sl   19:46   0:00 ollama serve
{"models":[]}---- last 30 lines of ollama.log ----
time=2026-01-29T19:46:47.497Z level=INFO source=routes.go:1631 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0

In [16]:
!pip -q install datasets requests tqdm
!sudo apt-get -qq update
!sudo apt-get -qq install -y pciutils lsb-release curl
!curl -fsSL https://ollama.com/install.sh | sh


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:
# Start ollama in the background
!nohup ollama serve > ollama.log 2>&1 &

# Quick sanity check: server should answer (may take a few seconds right after start)
!curl -s http://localhost:11434/api/tags | head

# Pull the model (about ~1.3GB)
!ollama pull llama3.2:1b


{"models":[]}


In [21]:
!hf version

huggingface_hub version: 0.36.0


In [22]:
!git config --global credential.helper store
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `scarlettyucs6501` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cac

In [26]:
!ls

llama_mmlu_eval.py	   llama_mmlu_eval_topic2.py  sample_data
llama_mmlu_eval_topic1.py  ollama.log


In [27]:
!time { python llama_mmlu_eval_topic1.py ; python llama_mmlu_eval_topic2.py ; }


Llama 3.2-1B MMLU Evaluation (Quantized)

Environment Check
âœ“ Running in Google Colab
âœ“ Platform: Linux (x86_64)
âš ï¸  No GPU detected - running on CPU
âœ“ Quantization disabled - loading full precision model
âœ“ Hugging Face authenticated

Configuration
Model: allenai/OLMo-2-0425-1B
Device: cpu
Quantization: None (full precision)
Expected memory: ~5 GB (FP32)
Number of subjects: 1


Loading model allenai/OLMo-2-0425-1B...
Device: cpu
tokenizer_config.json: 4.34kB [00:00, 5.33MB/s]
vocab.json: 1.61MB [00:00, 9.05MB/s]
merges.txt: 917kB [00:00, 5.73MB/s]
tokenizer.json: 7.14MB [00:00, 18.3MB/s]
special_tokens_map.json: 100% 125/125 [00:00<00:00, 853kB/s]
âœ“ Tokenizer loaded
Loading model (this may take 2-3 minutes)...
config.json: 100% 623/623 [00:00<00:00, 4.31MB/s]
2026-01-29 20:02:04.853824: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-29 20:02:05.279146: I external/local_xla/xla/tsl/cuda/cudar

In [2]:
!time { python llama_mmlu_eval_topic1.py & python llama_mmlu_eval_topic2.py & wait ; }



Llama 3.2-1B MMLU Evaluation (Quantized)
Llama 3.2-1B MMLU Evaluation (Quantized)


Environment Check
Environment Check
âœ“ Running in Google Colab
âœ“ Platform: Linux (x86_64)
âœ“ Running in Google Colab
âœ“ Platform: Linux (x86_64)
âœ“ GPU Available: Tesla T4
âœ“ GPU Memory: 15.83 GB
âœ“ Quantization disabled - loading full precision model
âš ï¸  No Hugging Face token found
Run: huggingface-cli login

Configuration
Model: allenai/OLMo-2-0425-1B
Device: cuda
Quantization: None (full precision)
Expected memory: ~2.5 GB (FP16)
Number of subjects: 1


Loading model allenai/OLMo-2-0425-1B...
Device: cuda
âœ“ GPU Available: Tesla T4
âœ“ GPU Memory: 15.83 GB
âœ“ Quantization disabled - loading full precision model
âš ï¸  No Hugging Face token found
Run: huggingface-cli login

Configuration
Model: allenai/OLMo-2-0425-1B
Device: cuda
Quantization: None (full precision)
Expected memory: ~2.5 GB (FP16)
Number of subjects: 1


Loading model allenai/OLMo-2-0425-1B...
Device: cuda
tokenizer_co